# IT Security - Sheet 4 "Authentication and Key Agreement"

**Total achievable points: 20**

**Released: 28.11.2024**

**Submission Deadline: 05.12.2024**

Group: 128

Names and matriculation numbers of **ALL** team members: Samuel Rode (445160), Nils Maasch (445796), Pau Azpeita Bergos (443428), Gereon Geuchen (445328), Ben-Jay Huckebrink (445219)

Format: John Doe (999999)

**Important Information**

The assignments have to be submitted by groups of 5 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`. We will then assign you to a submission group.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. In code cells, you have to remove `raise NotImplementedError()`. Please do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Please do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

Please **do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

In this exercise two additional packages are needed. The already known Cryptography package from the second assignment is needed. If not already installed you can install it using `pip install cryptography`. Additionally, another package is needed to interactively display graphs in this assignment. This package is called `plotly` and can be installed using `pip install plotly`. Please make sure to install the packages in the same environment as your jupyter notebook (`ipykernel`).

## Content of this Assignment

As we have learned a lot about RSA and asymmetric cryptography in the last assignment, we will now deepen our knowledge and implement an improved RSA backdoor in the first task. Afterwards we will deal with some more applications of asymmetric cryptography, especially about the Diffie-Hellman Key Agreement and Certificates. At the end of this exercise, you will again be given sample tasks.

## 1. Better RSA Backdoor (7 points)

In the last assignment you implemented a naive RSA backdoor. However, one drawback of this approach was that anyone who observed two public keys generated with this backdoor could exploit it as we fixed one of the primes.

### Task 1.1 (4 points)

Now, the smart-lock manufacturer BackDoor Locks Inc. wants to include a backdoor in the key generation module in one of their new locks. However, they want to make sure that only them can use the backdoor. Hence, they task you to implement this since they trust you as their local security expert regarding asymmetric cryptography.

As a preparation they already generated a key pair they want to use to secure the backdoor. Only someone with access to this key should be able to use this backdoor. This key pair is a RSA key pair given as `(e_man, N_man), d_man`.
Furthermore, they already included some primes `p_man` and `q_man` that you should use which were generated [randomly](https://xkcd.com/221/). These are given below as `p_man, q_man`.

In [28]:
# manufacturer key
e_man = 5
N_man = 10025419933826261
d_man = 2005083946714445

# two very random primes
p_man = 17
q_man = 19

from math import gcd

The first thing to do now is to implement a function which will generate a valid RSA key pair that incorporates this backdoor.

You need to implement the function `generate_backdoored_key(e_man: int, N_man: int, d_man: int, p: int, q: int) -> tuple[int, int, int]` which generates a new key pair `(e, N, d)`. This key should be based on the manufacturer's key (as given above) such that the manufacturer can use the "better backdoor" as introduced in the lecture on slide 30. Use the primes given, but still make sure that everything you calculate fulfills the needed properties. The function arguments and returns are as follows:
- `e_man: int`, `N_man: int`, `d_man: int`: the manufacturer's secret key (given above)
- `p`, `q`: the primes used to generate the new key (in this case given above)

Your function should return a tuple of the form `(e, N, d)` and `(0, 0, 0)` if e is not invertible mod `phi(n)`.

*Hint: This time, you are allowed to use the `gcd` function from the `math` module to calculate the greatest common divisor of two integers and the Python `pow` function to calculate the inverse of a integer modulo another integer (which uses the extended euclidean algorithm to achieve that). This [function](https://docs.python.org/3/library/math.html#math.gcd) is already imported in the cell above.*

In [ ]:
def generate_backdoored_key(e_man: int, N_man: int, d_man:int, p: int, q: int) -> tuple[int, int, int]:
    # YOUR CODE HERE
    N = p * q
    
    e = pow(p, e_man, N_man)
    
    phi = (p - 1) * (q - 1)
    if gcd(e, phi) != 1:
        return (0, 0, 0)
    
    d = pow(e, -1, phi)
    return (e, N, d)


In [3]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(generate_backdoored_key(e_man, N_man, d_man, p_man, q_man))

(1419857, 323, 17)


In [4]:
# This test just checks your solution

res = generate_backdoored_key(e_man, N_man, d_man, p_man, q_man)
assert type(res) == tuple
assert len(res) == 3
assert type(res[0]) == int
assert type(res[1]) == int
assert type(res[2]) == int

assert generate_backdoored_key(e_man, N_man, d_man, p_man, q_man) == (1419857, 323, 17)

In [5]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.2 (3 points)

All new smart locks of BackDoor Locks Inc. now use the implemented backdoor to generate their keying material.
From the service team there comes the request to be able to decrypt the exchanged messages between the lock and the controlling devices (like smartphones). This is especially helpful for customers who accidentally lost all their devices to unlock the smart lock.
Now, you are asked to implement this functionality using the backdoor.

For testing purposes of this new functionality, you are now given the following message `c` that was encrypted with your backdoored public key `(e, N)` you generated previously with the given primes `p_man, q_man`.

You need to implement the function `decrypt_backdoored_message(c: int, e: int, N: int, e_man: int, N_man: int, d_man: int) -> tuple[int, int]` that uses the knowledge of the manufacturer's key with `(e_man, N_man), d_man` to derive the private key `d` without having to generate the key yourself and then decrypt the given message `c`. The arguments and the returns are defined as follows:
- `c: int`: the encrypted message that needs to be decrypted
- `e: int, N:int`: the public key of the smart lock (generated in the previous task)
- `e_man: int, N_man: int, d_man: int`: the manufacturer's secret key

The function has to return a tuple consisting of the reconstructed private key `d` and the decrypted message `m` (decrypt the message `c` to `m`) in the form `(d, m)`.

In [6]:
c = 478

In [7]:
def decrypt_backdoored_message(c: int, e: int, N: int, e_man: int, N_man: int, d_man: int) -> tuple[int, int]:
    # YOUR CODE HERE
    p = pow(e, d_man, N_man)
    q = N // p 
    n = p*q
    phi = (p-1)*(q-1)
    d = pow(e, -1, phi)
    return (d, pow(c, d, n))

In [8]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [9]:
# This test just checks your solution

(e, N, d) = generate_backdoored_key(e_man, N_man, d_man, p_man, q_man)

res = decrypt_backdoored_message(c, e, N, e_man, N_man, d_man)

assert type(res) == tuple
assert len(res) == 2
assert type(res[0]) == int
assert type(res[1]) == int

assert res[0] == d

In [10]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 2. Diffie-Hellman Key Agreement (6 points)

Finally we turn away from RSA backdoors and towards another very important topic: the Diffie-Hellman Key Agreement.
In the lecture you heard about a secure and easy way to agree on a common key, this is called the Diffie-Hellman Key Agreement. In the following, we want to try to implement this. 

The usage of a Diffie-Hellman Key Agreement requires to have a prime `p` and a generator `g` of `Z*_p`. In this task, you can assume`p = 7` and the generator `g = 5` as given (in the cell below).

In [11]:
g = 5
p = 7

from random import choice

### Task 2.1 (1.5 point)

With this information, you can now implement the function `gen_dh_value(p: int, g: int) -> tuple[int, int]` which generates the public DH value of a communication party, based on `p` and `g`.

The arguments to the function are as follows:
- `p: int`: prime used during the dh generation
- `g: int`: the generator used for the generation of the public dh value

The function has to return a tuple of the form `(public_DH_value, rand)` where `rand` is the random value that you picked to create the public DH value `public_DH_value`.

*Use the `choice()` function to pick a random value. This [function](https://docs.python.org/3/library/random.html#random.choice) is already imported from the random package. You can just call `choice(...)` with a list of numbers the function should randomly choose from.*

In [12]:
def gen_dh_value(p: int, g: int) -> tuple[int, int]:    
    # YOUR CODE HERE
    rand = choice([i for i in range(2, p-1)])
    return (pow(g, rand, p), rand)

In [13]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [14]:
# This test just checks the output format of your solution

res = gen_dh_value(p, g)

assert type(res) == tuple
assert len(res) == 2
assert type(res[0]) == int
assert type(res[1]) == int

In [15]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.2 (1.5 point)

With the knowledge of the public DH value of the other communicating party, we can finally calculate the shared key. To do so, implement the function `gen_shared_key(other_pub_value: int, own_priv_value: int, p: int)`.
The arguments and returns of the function are as follows:
- `other_pub_value: int`: the public value of the other communicating party
- `own_priv_value: int`: the private random value of the current party
- `p: int`: the prime `p`

The function has to return the shared key as `int`.

In [16]:
def gen_shared_key(other_pub_value: int, own_priv_value: int, p: int) -> int:
    # YOUR CODE HERE
    return pow(other_pub_value, own_priv_value, p)

In [17]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [18]:
# This test just checks your solution

(pub_dh1, rand1) = gen_dh_value(p, g)
(pub_dh2, rand2) = gen_dh_value(p, g)
res2 = gen_shared_key(pub_dh1, rand2, p)
res1 = gen_shared_key(pub_dh2, rand1, p)

assert type(res1) == int
assert type(res2) == int
assert res1 == res2

In [19]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.3 (3 points)

Now, we get to our "simulation". If your implemented functions are correct, you should be able to help Alice and Bob in their key agreement!
You can test it yourself and play around with the values of `p` and `g`.

In [20]:
# "Simulation" - you do not need to implement something in this cell, but feel free to play around

color_code_A = '\x1b[1;31m'
color_code_B = '\x1b[1;32m'
color_end = '\x1b[0m'

p = 23
g = 5

# Bob and Alice both calculate their public and private values
A, a = gen_dh_value(p, g)
B, b = gen_dh_value(p, g)

# Alice sends their public DH value
print(f"{color_code_A}Alice: My public DH value is A = {A}.{color_end}")

# Bob can now calculate the shared key 
print("🧮 Bob calculates the shared key..")
bob_shared_key = gen_shared_key(A, b, p)
print(f"🔑 Bob uses the key K = {bob_shared_key} now.\n")

# Bob sends their public DH value 
print(f"{color_code_B}Bob: My public DH value is B = {B}.{color_end}")

# Alice can now calculate the shared key too
print("🧮 Alice calculates the shared key..")
alice_shared_key = gen_shared_key(B, a, p)
print(f"🔑 Bob uses the key K = {bob_shared_key} now.\n")

print(f"If you were successful, it should hold that {color_code_A}{alice_shared_key}{color_end} == {color_code_B}{bob_shared_key}{color_end}!")


Alice: My public DH value is A = 2.
🧮 Bob calculates the shared key..
🔑 Bob uses the key K = 13 now.

Bob: My public DH value is B = 6.
🧮 Alice calculates the shared key..
🔑 Bob uses the key K = 13 now.

If you were successful, it should hold that 13 == 13!


#### a) (0.5 points)

**Explain** why does the shared key change on different runs.

Because the two parties choose their own private value uniformly at random in the range ${{2,...,p-1}}$

#### b) (1.5 point)

**Name** the kind of attack to which this simple key agreement protocol is vulnerable to and **describe** how this can be done by an attacker.

Man-in-the-Middle-Attack. An Attacker can simply pretend that he is the other party and return his own public values to the two parties each. Both parties then assume that they share a key with the correct other party and do not realise that they share their key with the attacker.

#### c) (1 point)

How can we fix this vulnerability?

The two parties could also attach a digital signature to their messages, assuring that their messages cannot be replicated by the attacker.

## 3. Certificates (6 points)

### Task 3.1 (1 point)

As we now learned a lot about symmetric and asymmetric cryptography, we want to have a look at the differences. Especially, the number of necessary keys for different situations is interesting.

We now want to compare symmetric and asymmetric crypto in regard of number of keys used to secure communication between different parties.
As we want to visualize the differences effectively, we need an automatic way to compute the number of needed keys.

For this you have to implement the function `calc_needed_keys(n: int) -> tuple[int, int]` which computes the number keys needed to enable secure pairwise communication between `n` different parties.

Your function should return a tuple of the form `(x,y)` where `x` denotes the number of necessary keys in the symmetric case and `y` denotes the number of necessary keys in the asymmetric case. Additionally, you have to raise a `ValueError` if the input `n` is negative.

In [ ]:
def calc_needed_keys(n: int) -> tuple[int, int]:
    # YOUR CODE HERE
    if n < 0:
        raise ValueError
    return ((n*(n-1))//2, n)

In [ ]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [23]:
# This test just checks your solution

res = calc_needed_keys(2)

assert type(res) == tuple
assert len(res) == 2
assert type(res[0]) == int
assert type(res[1]) == int

assert res == (1,2)

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it


     ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
     -- ------------------------------------- 10.2/139.8 kB ? eta -:--:--
     -------- ---------------------------- 30.7/139.8 kB 435.7 kB/s eta 0:00:01
     ---------------- -------------------- 61.4/139.8 kB 469.7 kB/s eta 0:00:01
     ------------------------------- ---- 122.9/139.8 kB 722.1 kB/s eta 0:00:01
     ------------------------------------ 139.8/139.8 kB 692.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
     ------------------ ------------------- 102.4/214.4 kB 3.0 MB/s eta 0:00:01
     ------------------------------------ - 204.8/214.4 kB 2.1 MB/s eta 0:00:01
     -------------------------------------- 214.4/214.4 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
     ---- ----------------------------------- 0.3/2.3 MB 8.3 MB/s eta 0:00:


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\gereo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Now, you can use your implemented function to compare the amount of needed keys in both the symmetric and asymmetric case depending on the number of parties that want to communicate pairwise. Run the code and play around with the slider!

If your code works, you should be able to see and update the plot.

In [ ]:
# Again, you do not need to implement anything in this cell but only play around with it
import plotly.graph_objs as go
from ipywidgets import interact, widgets        
from plotly.subplots import make_subplots

def update_plot(num_parties):
    key_numbers_as = list()
    key_numbers_s = list()
    for i in range(2,num_parties+1):
        keys = calc_needed_keys(i)
        num_s_keys = keys[0]
        num_as_keys = keys[1]
        key_numbers_s.append(num_s_keys)
        key_numbers_as.append(num_as_keys)

    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.add_trace(go.Scatter(x=list(range(2,num_parties+1)), y=key_numbers_as, mode='lines+markers', name='Asymmetric Crypto'))
    fig.add_trace(go.Scatter(x=list(range(2,num_parties+1)), y=key_numbers_s, mode='lines+markers', name='Symmetric Crypto'))

    fig.update_layout(title='Number of Needed Keys vs. Number of Communicating Parties',
                     xaxis_title='Number of Communicating Parties',
                     yaxis_title='Number of Needed Keys',
    )

    fig.show()

# Set up slider
num_parties_slider = widgets.IntSlider(value=2, min=2, max=1000, step=1, description='Parties')

# Create interactive plot
interact(update_plot, num_parties=num_parties_slider)


This awesome visualization shows the benefits of using asymmetric cryptography when communicating with a lot of different parties. You can see that it quickly becomes a lot of keys that are needed for symmetric cryptography as each and every pair of communicating partners need their own key. In the case of asymmetric cryptography, the number of keys is equal to the number of parties and grows linearly. Therefore, an advantage of asymmetric cryptography is obviously that we only need to distribute the public key of a party to be able to securely communicate with that party instead of agreeing or exchanging a new key with each party.

One problem that remains is: how do we distribute those public keys in an authentic way? If someone would be able to place distribute his public key instead of the correct one, the attacker would be able to perform a person in the middle/man in the middle attack.
Here, certificates come into play! Certificates can help to distribute public keys in an authentic way. A so called certification authority creates a certificate that links the identity and the public key. To accomplish this, a digital signature of the certification authority guarantees the correctness of the certificate.

We will take a little peek into what information can be stored in a certificate. In the following, you are given a certificate in the [PEM](https://en.wikipedia.org/wiki/Privacy-Enhanced_Mail) format.

In [2]:
cert_str = """-----BEGIN CERTIFICATE-----
MIIGTzCCBDegAwIBAgIUH9/EC0Vf4GfPhyedTIeYK8P/Fh8wDQYJKoZIhvcNAQEL
BQAwgbYxCzAJBgNVBAYTAktZMQ8wDQYDVQQIDAZQZXJsaXMxDjAMBgNVBAcMBUFj
Y3JhMRwwGgYDVQQKDBNCYWNrRG9vciBMb2NrcyBJbmMuMR0wGwYDVQQLDBRCcmVh
Y2ggUmVzcG9uc2UgVGVhbTEbMBkGA1UEAwwSbG9ja3MuYmFja2Rvb3IuaW5jMSww
KgYJKoZIhvcNAQkBFh1yZWN5Y2xlYmluQGxvY2tzLmJhY2tkb29yLmluYzAeFw0y
NDExMjIxMzE2NDFaFw0yNTExMjIxMzE2NDFaMIG2MQswCQYDVQQGEwJLWTEPMA0G
A1UECAwGUGVybGlzMQ4wDAYDVQQHDAVBY2NyYTEcMBoGA1UECgwTQmFja0Rvb3Ig
TG9ja3MgSW5jLjEdMBsGA1UECwwUQnJlYWNoIFJlc3BvbnNlIFRlYW0xGzAZBgNV
BAMMEmxvY2tzLmJhY2tkb29yLmluYzEsMCoGCSqGSIb3DQEJARYdcmVjeWNsZWJp
bkBsb2Nrcy5iYWNrZG9vci5pbmMwggIiMA0GCSqGSIb3DQEBAQUAA4ICDwAwggIK
AoICAQCjrwM5P9YJFh7/Cffp0gdBugv4Il50dXmLGKkOET42GcC1xtAWdz5tG+Ai
2oz7Ro6XSBBqvsd7svb1gAFmo61h1atg+27IevBNA5zLP4G3P9tmJlGYUKGpRCDu
1FqbTV/ciQycPvys0m+9YQMiNPhvVsMCUrPkTC9b8di5e9OlLND8CkeNPcJASX/1
yds3j3SgnxbUTw4ds8lXSXIQOzgyvORpx9ohbb5nl2IwfuSJdcCFUVZ900fvfFP4
oAvUKGvQ6c1M0FF4Z3kLPCMZqcvPLHNRHFFBxuXg7gmM+4f2Sd6cJuMTqNW9hnvV
X0qcMWjR3R3glein5wthfpDAI0UHo39/OOpJMb5QznH7ju/1t/+iT2ETumtebp/Z
uuo4NL3yJ+Aub2Sza5X8A5D+VXFh1fUtQlYLyFnWIAJeRXGpBP+i74Em8PRJRHFw
Y9BkK7xye+Hw2TYWY8E7XXLxG51S+G6E4yK4oPGR0KRjMiyI+VH32JP79qp1FGwN
vS2NBbbZRNKKjLj5SVMu4r8/mwS/AA3yL48FaXG5kWfkcU7iKNWsKDTI43eHuKyH
WdKWTGvclOapnX6MOWX5T3iDDTtfdd8FgMiwt2FOVpms9i8AKYj6ONlvFeUGcowp
+MrXBgs5dJwtHsebuoXiqL7esZcUb9cVpRVfNzEAiUENf1LPFQIDAQABo1MwUTAd
BgNVHQ4EFgQUe4Buc5ylAwiP1oRJFFW1iJXJ4LYwHwYDVR0jBBgwFoAUe4Buc5yl
AwiP1oRJFFW1iJXJ4LYwDwYDVR0TAQH/BAUwAwEB/zANBgkqhkiG9w0BAQsFAAOC
AgEAVs0QaeTypyRjp895NfaJkrzNe4OMWysK37GpZozTEz27tN6JGWUafr+DVh06
CQ3K+CifuO7ODpKr37jUuTvUNVybkgZP3CSwuUsxvMnEPZggYb7m3CFFPfOBZhBU
IP1PC90pKq0xlFx5DCrrsqgPhe4xzFn0yde3o0IF41PRoZmLOEVGAqnem9G3os1i
AfPNQQq5/09RteKoSEtalfOa4NIltdWcq44VGJl7rAjbHo31bnaPSq7hGiu6skpE
HbfhYoIqVieVtLEk/awuZUI7fW4hfhY4b6KoXCONI2i/KpU7rASB5JtWwm9j/E0L
52lXak+3i+yu9bueWXyiaeb0oMrFg+N9pqKYpgtcZzHpgPThb8zHa7QiSTncB463
QRGr36p0eoS75DJtn9pMhDGsfMYCB7FoXZg6Vpltpz9GGh6JpZalGc42D0MC0kzT
mIuCkWhpLiHS0gXxz2QcGma77SFfNXo24jEx7DJr2sSS6196R9n7hFCGixEWG67N
zmNdXPENcDGunw4+O+EAgToOd80rvBDdxUcN4x4yZOLC8IyTSpQO/P1N1EaLGIQm
YDWRM7e351uAenx7PDHwCKVwqWIVuZdEkLj95COew57WG4qKBrGHRIRv3+4n4238
ndj6JAWxlczXYG7yq65kkhmNXNRAy0gw607Z78vl/jIJbgw=
-----END CERTIFICATE-----"""

### Task 3.2 (1 point)

For this task, we will be using the [cryptography](https://cryptography.io/en/latest/) module again. In the following cell are some imports, that we think could be part of a solution (you can import others from the module as needed). 
Also, we think the [datetime](https://docs.python.org/3/library/datetime.html) module might be of use and hence you are allowed to use it. It is also already imported.

In [3]:
# link https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/
from cryptography import x509
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.x509.name import Name
from datetime import datetime, timezone

First, implement the function `read_cert(pem_data: str) -> x509.Certificate` which takes a PEM-formatted string certificate as input (`pem_data`), reads it and returns it as an `x509.Certificate` object. For this you may want to have a look at [the documentation](https://cryptography.io/en/latest/x509/).

*Hint: If you need to encode a string as [bytes](https://docs.python.org/3/library/stdtypes.html#bytes), you can use the 'utf-8' encoding.*

In [4]:
def read_cert(pem_data: str) -> x509.Certificate:
    # YOUR CODE HERE
    data_bytes: bytes = bytes(pem_data, 'utf-8')
    return x509.load_pem_x509_certificate(data_bytes)

In [5]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(read_cert(cert_str))

<Certificate(subject=<Name(C=KY,ST=Perlis,L=Accra,O=BackDoor Locks Inc.,OU=Breach Response Team,CN=locks.backdoor.inc,1.2.840.113549.1.9.1=recyclebin@locks.backdoor.inc)>, ...)>


In [6]:
# This test just checks your solution

cert = read_cert(cert_str)

assert isinstance(cert, x509.Certificate)
assert cert.serial_number is not None

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.3 (0.5 points)

Now, we want to know the issuer of the certificate. Hence, implement a function `get_issuer(cert: x509.Certificate) -> Name` which takes the previously loaded `cert`, an `x509.Certificate`, as input and returns the issuer as `x509.name.Name`. 

*Hint: Don't be confused by the fact that your return value acts as a list (it is actually one!)*

In [7]:
def get_issuer(cert: x509.Certificate) -> Name:
    # YOUR CODE HERE
    return cert.issuer

In [8]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(get_issuer(read_cert(cert_str)))

<Name(C=KY,ST=Perlis,L=Accra,O=BackDoor Locks Inc.,OU=Breach Response Team,CN=locks.backdoor.inc,1.2.840.113549.1.9.1=recyclebin@locks.backdoor.inc)>


In [9]:
# This test just checks your solution

cert = read_cert(cert_str)
name = get_issuer(cert)

assert isinstance(name, x509.name.Name)
assert len(name) == 7

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.4 (0.5 points)

We also want to inspect the public key that this certificate informs us about. Implement the function `get_exponent(cert: x509.Certificate) -> int` which takes `cert`, an `x509.Certificate`, as input and returns the public exponent `e` of the public key in the certificate `cert` as `int`. 

In [10]:
def get_exponent(cert: x509.Certificate) -> int:
    # YOUR CODE HERE
    public_key = cert.public_key()
    return public_key.public_numbers().e

In [11]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(get_exponent(read_cert(cert_str)))

65537


In [12]:
# This test just checks your solution

cert = read_cert(cert_str)
exp = get_exponent(cert)

assert type(exp) == int

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.5 (1 point)

Of course, we also want to know about the length of the key. Implement the function `get_key_length(cert: x509.Certificate) -> int` which takes `cert`, an `x509.Certificate`, as input and returns the bit length of the key as `int`.

In [13]:
def get_key_length(cert: x509.Certificate) -> int:
    # YOUR CODE HERE
    key = cert.public_key()
    return key.key_size

In [14]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(get_key_length(read_cert(cert_str)))

4096


In [15]:
# This test just checks your solution

cert = read_cert(cert_str)
length = get_key_length(cert)

assert type(length) == int

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.6 (1 point)

Furthermore, we want to check whether the certificate is even valid for the current date (today). Hence, implement the function `check_validity(cert: x509.Certificate) -> bool` which takes `cert`, an `x509.Certificate`, as input and returns `True` if it is valid today (`datetime.today()`) and `False` otherwise.

*Hint: Have a look at all provided dates!*

In [ ]:
def check_validity(cert: x509.Certificate) -> bool:
    # YOUR CODE HERE
    # Yes, .now() instead of .today() _with timezone information_ because
    # otherwise, Python throws an error in the below comparison of dates
    cur_date = datetime.now(timezone.utc)

    if cur_date < cert.not_valid_before_utc:
        return False
    if cur_date > cert.not_valid_after_utc:
        return False
    return True

In [ ]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

In [17]:
# This test just checks your solution

cert = read_cert(cert_str)
valid = check_validity(cert)

assert type(valid) == bool
assert valid

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## Task 3.7 (1 point)

**Explain** what is difference between the above presented certificate and certificates used on most websites. Why is this certificate considered insecure?

The _difference_ to "normal" certificates & thus the _insecurity_ of the above certificate is due to it being a **self-signed certificate** (i.e. _issuer_ and "recipient" are the _same_, with the issuer _not being_ a certificate authority); in this case, both issuer & recipient are the "BackDoor Locks Inc." from Task 2.

This is considered insecure (and hence _not done_ on most websites) as when e.g. connecting to a website with a self-signed certificate, we **cannot check the legitimacy** of the certificate, as for all we know, some attacker could have just substituted this certificate _for their own self-signed_ certificate, thus making us believe we are connected to the "intended" communication partner when we are not.

## 4. Exam Example Tasks (0.5 points)

> Note: This task is awarded with 0.5 points only if it has been seriously addressed, regardless of whether the answer is correct.

In the following, multiple two party key establishment protocol will be introduced.

The key establishment protocola are based on the Diffie-Hellman Key Agreement. We assume the following for all of the following protocols:

- Alice (A), Bob (B), and Eve (E) each have a public/private key pair they can use to encrypt and (securely) sign messages.
- Alice, Bob, and Eve know each others public key
- We use $sig_X(m)$ to indicate that the message $m$ is signed (securely) using the private key of entity $X$
- The same holds for $enc_X(m)$ to indicate the encryption of message $m$ using the public key of $X$
- The signed messages are always verified by any receiving party, any invalid signature would lead to termination of the protocol
- The prime number $p$ and the generator $g \in \mathbb{Z}^*_g$ are public and known to all parties

### Task 4.1

The exchanged messages and actions of the first protocol are depicted in the following diagram:

![](./dh_sig.png)

**Answer** the questions below and **justify** your answer.

#### a)

Does an attacker (Eve), that is able to eavesdrop on every exchanged message, gets in possession of the key $K$ at the end?

No, she is not due to the hardness of the discrete logarithm problem. Although she knows A and g, she is not able to find a, same with b. Finding a or b is required to compute the key.

#### b)

Is a Person-in-the-middle-attack / Man-in-the-middle-addtack possible using this setting?

No, because the man-in-the-middle cannot sign messages correctly, thus Alice and Bob would know that there is an attacker who is trying to impersonate the other.

#### c)

From the lecture you know five properties for key establishment protocols: Entity Authentication, Implicit key authentication, Key freshness, Perfect forward secrecy, Protection against known keys.

State for each one, if this property holds or not. (No justification necessary)

Entity Authentication: No
Implicit key authentication: Yes
Key freshness: Yes
Perfect forward secrecy: No
Protection against known keys: No

#### d)

Could this key agreement be improved in any way to make it even more secure?

So far, there are no session keys. The key is established in the beginning and then used for the whole communication. The use of session keys would be an improvement: After a certain amount of messages, there has to be a new key agreement process, establishing a new key between the two parties. This would lead to an improvement and Perfect Forward Secrey as well as Protection Against Known Keys. Sending a certificate alongside the messages would also provide authentication.

### Task 4.2

In this task we have a look at another Diffie-Hellman Key Agreement protocol.

The exchanged messages and actions of the first protocol are depicted in the following diagram:

![](./dh_auth.png)

#### a)

For each of the message components (e.g. $\text{sig}_A(A)$, $A$, ...) **explain** what they are used for.

A: Public DH-Value of A, cert_A: Certificate, stating that A truly belongs to Alice, sig_A(A): Alice's signature, so B knows that the message has indeed been send by Alice
For Bob's message, it's completely the same, just the other way around.

#### b)

From the lecture you know five properties for key establishment protocols: Entity Authentication, Implicit key authentication, Key freshness, Perfect forward secrecy, Protection against known keys.

State for each one, if this property holds or not. (No justification necessary)

Entity Authentication: No
Implicit key authentication: Yes
Key freshness: Yes
Perfect forward secrecy: No
Protection against known keys: No

#### c)

Is this approach more secure than the previous one? Is an attack still possible? **Justify** your answer.

Yes, it is better, because there is now authentication due to the use of the certificates. An attack is still possible because a man-in-the-middle could intercept the messages, once again establish two seperate key exchanges with Alice and Bob and now send cert_A and cert_B.

## 5. Feedback (0.5 points)

> Note: This task is awarded with 0.5 points only if there is feedback in any form to this exercise. It is also okay to state what was especially nice or rewarding. Literally just writing "anything" is not enough. Any feedback will improve the assignments.

You can not be stopped - you finished one assignment more! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write some feedback.

This exercise was, in our opinion, a bit stale: We get that depending on the contents of the lecture, some topics are more suited to providing "good material" for coding exercises than others (e.g. two sheets with RSA backdoors on them shows that RSA is more well suited to this exercise system than other topics), but having one entire exercise (exercise 3) just dedicated to basically scrolling through the documentation of the `cryptography` module & implementing that is not really our favourite type of work. Instead, we would have liked exercise 3 being more like exercise 2, with maybe some coding tasks, but then more text-based/written tasks to compensate for the lesser amount of coding tasks.

**This assignment was brought to you by**

![](./logo.png)

BackDoor Locks Inc. *Innovation Meets... Convenient Entry.*

*Logo generated with the help of AI*